In [367]:
import pandas as pd
from numpy import sqrt,mean,log,diff
import QuantLib as ql
from pandas_datareader.data import Options
import pandas_datareader.data as web
import datetime 

In [368]:
opt = Options('spy', 'yahoo')

In [369]:
expiration_dates = [ql.Date(i.day, i.month, i.year) for i in opt.expiry_dates]
expiry_index = 14 # choose the contracts expire in 4 months
data = opt.get_call_data(expiry=opt.expiry_dates[expiry_index])
strikes = list(data.index.get_level_values('Strike'))
premium = list(data['Last'])

In [534]:
opt.expiry_dates[expiry_index]

datetime.date(2017, 11, 17)

In [370]:
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()
calculation_date = ql.Date(opt._quote_time.day,opt._quote_time.month,opt._quote_time.year)
spot = opt.underlying_price
ql.Settings.instance().evaluationDate = calculation_date
dividend_yield = ql.QuoteHandle(ql.SimpleQuote(0.0))
risk_free_rate = 0.01
dividend_rate = 0.0
flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count))
dividend_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count))

In [518]:
# dummy parameters
initial_var = 0.2; rate_reversion = 0.5; long_term_var = 0.2; corr = -0.75; vol_of_vol = 0.2;
# initial_var = 0.2; rate_reversion = 0.15; long_term_var = 0.6; corr = -0.75; vol_of_vol = 0.2;
process = ql.HestonProcess(flat_ts, dividend_ts, 
                           ql.QuoteHandle(ql.SimpleQuote(spot)), 
                           initial_var, rate_reversion, long_term_var, vol_of_vol, corr)
model = ql.HestonModel(process)
engine = ql.AnalyticHestonEngine(model) 

In [519]:
heston_helpers = []
date = expiration_dates[expiry_index]
for j, s in enumerate(strikes):
    t = (date - calculation_date)
    p = ql.Period(t, ql.Days)
    sigma = premium[j]
    helper = ql.HestonModelHelper(p, calendar, spot, s, 
                                  ql.QuoteHandle(ql.SimpleQuote(sigma)),
                                  flat_ts, 
                                  dividend_ts)
    helper.setPricingEngine(engine)
    heston_helpers.append(helper)

In [520]:
lm = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
model.calibrate(heston_helpers, lm, 
                 ql.EndCriteria(500, 50, 1.0e-8,1.0e-8, 1.0e-8))
long_term_var, rate_reversion, vol_of_vol, corr, initial_var = model.params()

In [521]:
print "long_term_var = %f, rate_reversion = %f, vol_of_vol = %f, corr = %f, initial_var = %f" % (long_term_var, rate_reversion, vol_of_vol, corr, initial_var)

long_term_var = 0.191762, rate_reversion = 0.000001, vol_of_vol = 0.215442, corr = -0.817388, initial_var = 0.198778


In [522]:
def mc_heston(option_type,S0,K,T,initial_var,long_term_var,rate_reversion,vol_of_vol,corr,r,num_reps,steps):
    """
    option_type:    'p' put option 'c' call option
    S0:              the spot price of underlying stock
    K:              the strike price
    T:              the maturity of options
    initial_var:    the initial value of variance
    long_term_var:  the long term average of price variance
    rate_reversion: the mean reversion rate for the variance
    vol_of_vol:     the volatility of volatility(the variance of the variance of stock price)
    corr:           the correlation between the standard normal random variables W1 and W2
    r:              the risk free rate
    reps:           the number of repeat for monte carlo simulation
    steps:          the number of steps in each simulation
    """
    delta_t = T/float(steps)
    payoff = 0
    for i in range(num_reps):
        vt = initial_var
        log_st = log(S0)
        for j in range(steps):
            w1 = np.random.normal(0, 1)
            w2 = corr*w1+sqrt(1-corr**2)*np.random.normal(0, 1)
            vt = (sqrt(vt) + 0.5 * vol_of_vol * sqrt(delta_t) * w1)**2  \
                 - rate_reversion * (vt - long_term_var) * delta_t \
                 - 0.25 * vol_of_vol**2 * delta_t
            if vt < 0: vt = 0.00
            log_st = log_st + (r - 0.5*vt)*delta_t + sqrt(vt)*sqrt(delta_t)*w2
        st = e**(log_st)
        if option_type == 'c':
                payoff += max(st - K, 0)
        elif option_type == 'p':
                payoff += max(K - st, 0)
        
    return (payoff/float(num_reps)) * (exp(-r*T))

In [523]:
mc_heston('c',spot,strikes[20],t/365.0,initial_var,long_term_var,rate_reversion,vol_of_vol,corr,0.01,100,1000)

44.383434958491186

In [524]:
premium[20]

36.119999999999997

In [525]:
heston = []

for i in range(len(strikes)):
    heston.append(mc_heston('c',spot,strikes[i],t/365.0,initial_var,long_term_var,rate_reversion,vol_of_vol,corr,0.01,100,100))
    

In [529]:
diff = [(heston[i]-premium[i])**2 for i in range(len(premium))]

In [530]:
sum(diff)

8885.1013305184661

In [531]:
zip(heston, premium)

[(110.46562812345368, 127.83),
 (110.6988943943971, 112.12),
 (55.926776121382609, 55.710000000000001),
 (52.981820291517408, 54.030000000000001),
 (54.164283840319435, 53.340000000000003),
 (54.991136360501223, 52.719999999999999),
 (41.216283956629468, 51.090000000000003),
 (47.687719919883207, 50.340000000000003),
 (44.545781271634148, 49.32),
 (47.563439672575591, 48.340000000000003),
 (44.635711823762009, 47.340000000000003),
 (46.252466035738571, 46.170000000000002),
 (47.574492181620364, 45.18),
 (41.653315658070859, 43.310000000000002),
 (47.604806657885561, 43.229999999999997),
 (40.894480554924115, 42.799999999999997),
 (38.539091518531059, 42.25),
 (43.303139080062273, 40.829999999999998),
 (36.234352000924453, 39.880000000000003),
 (39.257917188012463, 38.469999999999999),
 (44.853854689174561, 36.119999999999997),
 (39.640305408524711, 36.310000000000002),
 (34.129177338407089, 36.359999999999999),
 (36.438197093590269, 35.5),
 (39.905750087595855, 34.130000000000003),
 (3